In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 41.8 MB/s eta 0:00:00


In [ ]:
from rdkit import Chem, DataStructs
from rdkit.Chem import PandasTools, AllChem
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
file_name = "test.xls"

In [ ]:
df = pd.read_escel(f"./{file_name}")
df.head()

In [ ]:
#realize df describe to analyse data

In [ ]:
PandasTools.AddMoleculeColumnToFrame("DATAFRAME INSTEAD OF STRING", "SMILES", "Molecule")

In [ ]:
!python -m pip install git+https://github.com/EBjerrum/molvecgen


  Cloning https://github.com/EBjerrum/molvecgen to /tmp/pip-req-build-923jsg1r
  Running command git clone --filter=blob:none --quiet https://github.com/EBjerrum/molvecgen /tmp/pip-req-build-923jsg1r
  Resolved https://github.com/EBjerrum/molvecgen to commit f81d5aade18bea60882f5845877f6283366bbe91
  Preparing metadata (setup.py) ... done
  Created wheel for molvecgen: filename=molvecgen-0.1-py3-none-any.whl size=11357 sha256=bc46d6fd6d41156855c0e20e5791c0d9612ee4e3accbda8f3169742c7e74949f
  Stored in directory: /tmp/pip-ephem-wheel-cache-3uea1zym/wheels/1c/de/14/fd9892b48271e63e56b00c7015c754ae6189e258fa313a7c63
Successfully built molvecgen


In [ ]:
from molvecgen.vectorizers import SmilesVectorizer

In [ ]:
smivec = SmilesVectorizer(pad=1, leftpad=True, canonical=False, augment=True)
smivec.fit(df.Molecule.values)

In [ ]:
y = df[target]
X = df.Molecule.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)
scaler = StandardScaler()
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)
y_val = scaler.transform(y_val)

In [ ]:
class SMILESMolDataset(Dataset):
  def __init__(self, molecules, y, vectorizer):
    self.molecules = molecules
    self.y = y
    self.vectorizer = vectorizer
  def __len__(self):
    return len(self.molecules)
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    mols = self.molecules[idx]

    sample = self.vectorizer.transform([mols])[0]
    label = self.y[idx]
    return sample, label

In [ ]:
train_dataset = SMILESMolDataset(X_train, y_train, smivec)

In [ ]:
device = torch.device("cuda:0")

cuda:0


In [ ]:
X_val_t = smivec.transform(X_val, canonical=False)
X_val_t = torch.tensor(X_val_t, device=device).float()
y_val_t = torch.tensor(y_val, device=device).float()
y_val_t = torch.nan_to_num(y_val_t, nan=torch.nanmean(y_val_t))

In [ ]:
class NeuralNetworkMol(nn.Module):
  def __init__(self, dims, lstm_size, hidden_size, dropout_rate, out_size) -> None:
    super(NeuralNetworkMol, self).__init__()

    length = dims[0]
    number_tokens = dims[1]

    self.lstm = nn.GRU(input_size = number_tokens, hidden_size=lstm_size, num_layers=1, batch_first=True, bidirectional=False, dropout=0.2)
    #model

  def forward(self, x):
    # forward for netwrok